<img src="https://kaggle2.blob.core.windows.net/competitions/kaggle/4699/logos/front_page.png"/>

## <span style="color:blue;">Prudential Life Insurance Assessment - v2 XGBoost</span>

## Import Packages

In [4]:
using DataFrames
using XGBoost

## Load Data

In [41]:
train = readtable("data/train_v1.csv")
test = readtable("data/test_v1.csv");

## Set Features and Output

In [42]:
label    = :Response
labels   = length(Set(train[label]))
features = setdiff(names(test), [:id]);

## Prepare Training

In [43]:
function split_train_val(df; train_size=.85, random_state=1)
    
    srand(random_state)
    
    nrows, ntraining_rows = size(df, 1), round(Int, size(df, 1) * train_size)
    indexes               = shuffle(collect(1:nrows))
    train                 = df[indexes[1:ntraining_rows], :]
    validation            = df[indexes[ntraining_rows+1:end], :]
    
    return train, validation
end

split_train_val (generic function with 1 method)

In [45]:
X_train, X_val = split_train_val(train, train_size=.85, random_state=1)

train_x = Array{Float64,2}(X_train[:, features])
train_y = Array{Float64,1}(X_train[label]) - 1
val_x   = Array{Float64,2}(X_val[:, features])
val_y   = Array{Float64,1}(X_val[label]) - 1
test_x  = Array{Float64,2}(test[:, features]);

In [47]:
dtrain  = DMatrix(train_x, label=train_y)
dval    = DMatrix(val_x, label=val_y);

## Train Data

In [49]:
num_rounds = 40
watchlist  = [(dtrain, "train"), (dval, "eval")]
metrics    = ["merror", "mlogloss"]
params     = Dict("objective"         => "multi:softmax",
                   "booster"          => "gbtree",
                   "eta"              => .1,
                   "alpha"            => .5,
                   "gamma"            => .0,
                   "max_depth"        =>  5,
                   "colsample_bytree" => .5,
                   "min_child_weight" =>  10,
                   "subsample"        => .5,
                   "seed"             =>  1)

println("Training Base Model...")
tic()
model = XGBoost.xgboost(dtrain, num_rounds, param=params, metrics=metrics,
                        num_class=labels, watchlist=watchlist)
toc()

Training Base Model...


[1]	train-merror:0.517930	train-mlogloss:1.989812	eval-merror:0.510273	eval-mlogloss:1.988872
[2]	train-merror:0.481277	train-mlogloss:1.906776	eval-merror:0.474795	eval-mlogloss:1.905650
[3]	train-merror:0.467369	train-mlogloss:1.840379	eval-merror:0.463680	eval-mlogloss:1.839365
[4]	train-merror:0.458553	train-mlogloss:1.786465	eval-merror:0.455821	eval-mlogloss:1.785486
[5]	train-merror:0.454234	train-mlogloss:1.739917	eval-merror:0.450095	eval-mlogloss:1.739334
[6]	train-merror:0.450826	train-mlogloss:1.696697	eval-merror:0.446390	eval-mlogloss:1.696502
[7]	train-merror:0.449677	train-mlogloss:1.662524	eval-merror:0.448075	eval-mlogloss:1.662721
[8]	train-merror:0.444684	train-mlogloss:1.624432	eval-merror:0.443584	eval-mlogloss:1.624744
[9]	train-merror:0.443733	train-mlogloss:1.591114	eval-merror:0.442686	eval-mlogloss:1.591478
[10]	train-merror:0.439256	train-mlogloss:1.560969	eval-merror:0.441338	eval-mlogloss:1.561810
[11]	train-merror:0.439830	train-mlogloss:1.533383	eval-mer

elapsed time: 58

58.964680579

.964680579 seconds


## Prediction Test

In [51]:
yhat = XGBoost.predict(model, test_x) + 1;

## Generate Submission File

In [56]:
submission_df = DataFrame(Id=test[:Id], Response=Array{Int,1}(yhat))
writetable("data/submissions/submission_v1_xgb_nr40_e1_a5_g0_md5_c5_mcw10_s5.csv", submission_df);

## Kaggle Results

In [13]:
Best Score: <span style="color:blue;">0.</span> [v1]

- v1 **XGB** (NR50 E1 A5 G0 MD9 C5 MCW10 s5): **0.** 

LoadError: LoadError: syntax: extra token "Score" after end of expression
while loading In[13], in expression starting on line 1